In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [2]:
data = pd.read_csv('data0.csv')
data["date"] = pd.to_datetime(data["date"]) 
date_begin=pd.to_datetime('2021-01-01')
date_end=pd.to_datetime('2022-12-31')
data=data[(data['date']>=date_begin)&(data['date']<=date_end)]

prediction=pd.read_csv('problem1_prediction.csv')
prediction['ds'] = pd.to_datetime(prediction['ds'])


date_str='2023-01-01'
date_step=31
new_edge_value=1000
load_value=10000
create_value=1000
res_value=100000
step=1
max_edge_flow=505833

data.head()
# print(date+ pd.Timedelta(days=1))

,place1,place2,date,num
0,DC3,DC5,2021-01-01,3
1,DC3,DC10,2021-01-01,4
2,DC3,DC14,2021-01-01,4
3,DC5,DC3,2021-01-01,41
4,DC5,DC9,2021-01-01,3


In [3]:
def cost_func(u,v,not_used,current_flow,max_flow,not_created):
    load_rate=current_flow/max_flow
    res_flow=max_flow-current_flow
    if u=='s' or v=='t':
        return 0
    if '-pre' in u or '-next' in v:
        return 0
    return np.ceil(not_used*new_edge_value +load_rate*load_value+res_value/(res_flow+1)+not_created*create_value)

In [4]:
result_all_list=[]
date=pd.to_datetime(date_str)
for cnt in range(date_step):
    
    unique_points=pd.concat([data['place1'],data['place2']],ignore_index=True).drop_duplicates()
    point_info=pd.DataFrame(columns=['place','res','max_flow','in_flow','out_flow'])
    for index, value in unique_points.iteritems():
        if value=='DC9':
            continue
        in_flow=prediction[(prediction['ds']==date)&(prediction['place2']==value)&(prediction['place1']!='DC5') ]['yhat'].sum( )
        out_flow=prediction[(prediction['ds']==date)&(prediction['place1']==value)&(prediction['place2']!='DC5')]['yhat'].sum( )
        res=out_flow-in_flow
        
        max_in_sum_1_temp=data[(data['place2']==value)].groupby('date').sum()
        max_in_sum_1=max_in_sum_1_temp['num'].max()
        max_in_sum_2_temp=prediction[(prediction['place2']==value)][['yhat','ds']].groupby('ds').sum()
        max_in_sum_2=max_in_sum_2_temp['yhat'].max()
        if np.isnan(max_in_sum_1):
            max_in_sum_1=0
        if np.isnan(max_in_sum_2):
            max_in_sum_2=0
        max_in_sum=max(max_in_sum_1,max_in_sum_2)
        
        max_out_sum_1_temp=data[(data['place1']==value)].groupby('date').sum()
        max_out_sum_1=max_out_sum_1_temp['num'].max()
        max_out_sum_2_temp=prediction[(prediction['place1']==value)].groupby('ds').sum()
        max_out_sum_2=max_out_sum_2_temp['yhat'].max()
        if np.isnan(max_out_sum_1):
            max_out_sum_1=0
        if np.isnan(max_out_sum_2):
            max_out_sum_2=0
        max_out_sum=max(max_out_sum_1,max_out_sum_2)
        
        point_info.loc[index]=[value,res,max(max_in_sum,max_out_sum),in_flow,out_flow]
    point_info=point_info.reset_index()
    point_info.head()
    
    
    unique_routes = data[['place1', 'place2']].drop_duplicates()
    edge_info=pd.DataFrame(columns=['max_flow','place1','place2','current_flow','is_new_flow'])
    for index, row in unique_routes.iterrows():
        place1,place2=row['place1'],row['place2']
        if 'DC9' in [place1,place2]:
            continue
        max_flow_1=data[(data['place1']==place1) & (data['place2']==place2)]['num'].max()
        max_flow_2=prediction[(prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].max()
        max_flow=max(max_flow_1,max_flow_2)
        current_flow=prediction[(prediction['ds']==date) & (prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].values[0]
        edge_info.loc[index]=[max_flow,place1,place2,current_flow,False]

    nodes = set(edge_info['place1']).union(set(edge_info['place2']))
    node_combinations = list(itertools.combinations(nodes, 2))

    # Find missing edges
    existing_edges = set(zip(edge_info['place1'], edge_info['place2']))
    missing_edges = [edge for edge in node_combinations if edge not in existing_edges and edge[::-1] not in existing_edges]

    # Add missing edges to edge_info
    for place1, place2 in missing_edges:
        edge_info.append({'max_flow': max_edge_flow, 'place1': place1, 'place2': place2, 'current_flow': 0,'is_new_flow':True}, ignore_index=True)
        edge_info.append({'max_flow': max_edge_flow, 'place1': place2, 'place2': place1, 'current_flow': 0,'is_new_flow':True}, ignore_index=True)

    edge_info=edge_info.reset_index()
    edge_info.head() 
    
    
    # 创建有向图
    G = nx.DiGraph()

    # 添加节点
    G.add_node('s')  # 源节点
    G.add_node('t')  # 汇节点 

    for index, row in point_info.iterrows():
        point,max_flow,res,in_flow,out_flow=row['place'],row['max_flow'],row['res'],row['in_flow'],row['out_flow']
        G.add_node(point)
        G.add_node(point+'-pre')
        G.add_node(point+'-next')
        
        G.add_edge(point+'-pre', point, capacity=max(max_flow-in_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))
        G.add_edge(point, point+'-next', capacity=max(max_flow-out_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))


    # print(G.nodes())
    # 添加边
    for index, row in edge_info.iterrows():
        u,v,max_flow,current_flow,is_new_flow=row['place1'],row['place2'],row['max_flow'],row['current_flow'],row['is_new_flow']
        cost=cost_func(u,v,True,current_flow,max_flow,is_new_flow)
        G.add_edge(u+'-next', v+'-pre', capacity=max(0,max_flow-current_flow), weight=cost,flow=current_flow,origin_capacity=max_flow)

    for index, row in point_info.iterrows():
        place,res,max_flow=row['place'],row['res'],row['max_flow']
        if res>0:
            G.add_edge('s', place+'-pre', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)
        else:
            G.add_edge(place+'-next', 't', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)


    flow_to_DC9=prediction[(prediction['ds']==date) & (prediction['place2']=='DC9')]['yhat'].sum()
    flowDict=nx.max_flow_min_cost(G, 's', 't', capacity='capacity', weight='weight')
    max_flow=sum(flowDict[node1][node2] for node1 in flowDict for node2 in flowDict[node1])
    print(flow_to_DC9,max_flow)
    min_cost_list=[]
    flow_dict_list=[]

    add_num,i=1,0
    while i<flow_to_DC9:
        # print(f'{i}/{flow_to_DC9}')
        i+=add_num
        if add_num<step:
            add_num+=1
        G.nodes['s']['demand']=-add_num
        G.nodes['t']['demand']=add_num
        flowDict = nx.min_cost_flow(G, demand="demand", capacity='capacity', weight='weight')
        # min_cost=nx.cost_of_flow(G,flowDict)
        # min_cost_list.append(min_cost)
        flow_dict_list.append(flowDict)
        # print(min_cost)
        for u, v in G.edges():
            if flowDict[u][v]!=0:
                # print(u,v,flowDict[u][v])
                G[u][v]['capacity']-=flowDict[u][v]
                G[u][v]['flow']  += flowDict[u][v]
                G[u][v]['weight']=cost_func(u,v,False,G[u][v]['flow'],G[u][v]['origin_capacity'],False)
                # print(u,v,G[u][v]['weight'])
                
                
    result_all_list.append(flow_dict_list)
    date=date+pd.DateOffset(days=1)
    print(cnt)

    
    

62 27861127
0
67 29552845
1
83 28252798
2
79 26760552
3
80 24809169
4
81 23556155
5
83 22694371
6
79 22167600
7
80 20989983
8
96 20393569
9
87 19728779
10
87 18703727
11
90 17895129
12
80 18152778
13
83 18335618
14
82 17281341
15
98 17172400
16
91 16829064
17
88 16362563
18
97 16591120
19
89 15757350
20
88 16051084
21
95 15860635
22
111 16043251
23
103 16116604
24
107 16025583
25
110 16352077
26
108 16205602
27
110 16596125
28
121 16635570
29
142 16922619
30


In [5]:
df_list = []
date=pd.to_datetime(date_str)
for index,flow_dict_list in enumerate(result_all_list):
    result_dict={}
    for d in flow_dict_list:
        for k1, v1 in d.items():
            if k1 not in result_dict:
                result_dict[k1] = {}
            for k2, v2 in v1.items():
                result_dict[k1][k2] = result_dict[k1].get(k2, 0) + v2
    result_dict_new={}          
    for k1,v1 in result_dict.items():
        for k2,v2 in v1.items():
            if v2!=0:
                k1=k1.replace('-pre','').replace('-next','')
                k2=k2.replace('-pre','').replace('-next','')
                if k1==k2:
                    continue
                if k1 not in result_dict_new:
                    result_dict_new[k1] = {}
                result_dict_new[k1][k2] = v2
    result_dict=result_dict_new
    for k1,v1 in result_dict.items():
        for k2,v2 in v1.items():
            if not(k1=='s' or k2=='t'):
                max_flow=edge_info[(edge_info['place1']==k1) & (edge_info['place2']==k2)]['max_flow'].values[0]
                origin_flow=edge_info[(edge_info['place1']==k1) & (edge_info['place2']==k2)]['current_flow'].values[0]
                is_new_flow=edge_info[(edge_info['place1']==k1) & (edge_info['place2']==k2)]['is_new_flow'].values[0]
                result_dict[k1][k2] = [v2,(v2+origin_flow)/max_flow,max_flow,origin_flow,is_new_flow]
            else:
                result_dict[k1][k2] = [v2,0,0,0,False]
    df_list.append(pd.DataFrame([(k1, k2, v2[0],v2[1],v2[2],v2[3],date,v2[4]) for k1,v1 in result_dict.items() for k2,v2 in v1.items()], columns=['place1', 'place2', 'flow','load_rate','max_flow','origin_flow','date','is_new_create']))
    date=date+pd.DateOffset(days=1)
    
df = pd.concat(df_list)
df.to_csv('problem3_all.csv', index=True)  